## Загрузка бибилиотек

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Загрузка и подготовка данных

Посмотрим на примере данных по оттоку клиентов из телеком-компании.

> **Важно**: XGBoost поддерживает только числовые признаки. Поэтому текстовые признаки требуют предобработки

Для обучения с Xgboost данные должны быть представлены в виде объекта `DMatrix`, предназначенного для представления разреженных данных. Его можно инициализировать через:
- текстовый формат libsvm,
-  Numpy 2D array (чаще всего)
- бинарный буфер-файл XGBoost 

**Загрузим данные и осуществим минимальную предобработку.**

In [2]:
df = pd.read_csv('telecom_churn.csv')

In [3]:
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


**Штаты просто занумеруем, а признаки International plan (наличие международного роуминга), Voice mail plan (наличие голосовой почтыы) и целевой Churn сделаем бинарными.**

In [4]:
state_enc = LabelEncoder()
df['State'] = state_enc.fit_transform(df['State'])
df['International plan'] = (df['International plan'] == 'Yes').astype('int')
df['Voice mail plan'] = (df['Voice mail plan'] == 'Yes').astype('int')
df['Churn'] = (df['Churn']).astype('int')

**Разделим данные на обучающую и тестовую выборки в отношении 7:3. Инициализируем соотв. объекты DMatrix dtrain и dtest.**

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Churn', axis=1), df['Churn'],
                                                    test_size=0.3, random_state=42)
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

**Посмотрим на статистику полученных объектов:**

In [6]:
print("Train dataset contains {0} rows and {1} columns".format(dtrain.num_row(), dtrain.num_col()))
print("Test dataset contains {0} rows and {1} columns".format(dtest.num_row(), dtest.num_col()))

Train dataset contains 2333 rows and 19 columns
Test dataset contains 1000 rows and 19 columns


In [7]:
print("Train possible labels: ")
print(np.unique(dtrain.get_label()))

print("\nTest possible labels: ")
print(np.unique(dtest.get_label()))

Train possible labels: 
[ 0.  1.]

Test possible labels: 
[ 0.  1.]


### Инициализация параметров

- бинарная классификация (`'objective':'binary:logistic'`)
- ограничим глубину деревьев (`'max_depth':3`)
- не хотим лишнего вывода (`'silent':1`)
- проведем 10 итераций бустинга
- шаг градиентного спуска довольно большой (`'eta':1`) - алгоритм будет обучаться быстро и "агрессивно" (лучше результаты будут, если уменьшить eta и увеличить число итераций)


In [8]:
params = {
    'objective':'binary:logistic',
    'max_depth': 3,
    'silent': 1,
    'eta': 1
}

num_rounds = 10

### Обучение классификатора
Тут мы просто передаем слоавть параметров, данные и число итераций.

In [9]:
xgb_model = xgb.train(params, dtrain, num_rounds)

**С помощью `watchlist` отслеживать качество алгоритма на тестовой выборке для каждой итерации.**

In [10]:
watchlist  = [(dtest,'test'), (dtrain,'train')] # native interface only
xgb_model = xgb.train(params, dtrain, num_rounds, watchlist)

[0]	test-error:0.1	train-error:0.091299
[1]	test-error:0.078	train-error:0.070296
[2]	test-error:0.059	train-error:0.051865
[3]	test-error:0.062	train-error:0.04715
[4]	test-error:0.061	train-error:0.049721
[5]	test-error:0.057	train-error:0.044578
[6]	test-error:0.054	train-error:0.040291
[7]	test-error:0.056	train-error:0.03772
[8]	test-error:0.053	train-error:0.036434
[9]	test-error:0.054	train-error:0.035577


### Прогнозы для тестовой выборки

In [11]:
preds_prob = xgb_model.predict(dtest)

**Посчитаем долю правильных ответов алгоритма на тестовой выборке.**

In [13]:
predicted_labels = preds_prob > 0.5
print("Accuracy and F1 on the test set are: {} and {}".format(
    round(accuracy_score(y_test, predicted_labels), 3),
    round(f1_score(y_test, predicted_labels), 3)))

Accuracy and F1 on the test set are: 0.946 and 0.789
